# Fake Text

- ### Imports

In [1]:
import nltk
import os
import pandas as pd
import csv
import numpy as np
import codecs
import re
import collections
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.feature_extraction import DictVectorizer
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from collections import Counter
from nltk.tokenize import punkt
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import feature_extraction
from sklearn import preprocessing

from unidecode import unidecode
import unicodedata

from nltk.tokenize import sent_tokenize
import string
from nltk.stem.snowball import FrenchStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize.texttiling import TextTilingTokenizer
from nltk.collocations import *
import unicodedata
from nltk.stem import WordNetLemmatizer

In [2]:
#load data
all_news = pd.read_csv("data/fake.csv",sep=",")
print("Types and counts of stories", all_news.groupby(["type"]).size())

('Types and counts of stories', type
bias            443
bs            11492
conspiracy      430
fake             19
hate            246
junksci         102
satire          146
state           121
dtype: int64)


In [3]:
all_news.head()

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


In [4]:
le = preprocessing.LabelEncoder()
label = le.fit_transform(all_news['type'])
#le.transform(["tokyo", "tokyo", "paris"]) 

all_news['target']=label

- ### Def functions

In [5]:
def word_stemmer(mots):
    snowball = SnowballStemmer('english') # for english use Porter stem algo
    preprocessed_docs = []
    for mot in mots:
        preprocessed_docs.append(snowball.stem(mot))   
    return preprocessed_docs

def word_lemmatizer(mots): 
    lemmat = WordNetLemmatizer()
    preprocessed_docs = []
    for mot in mots:
        preprocessed_docs.append(lemmat.lemmatize(mot))   
    return preprocessed_docs 


def clean_text(txt):
    e=str(txt)
    e=unicode(e,'utf-8')
    e=unicodedata.normalize('NFKD', e).encode('ascii', 'ignore')
    e=re.sub("[^a-zA-Z]", " ", e)
    return e

def chunk_mot(phrases):
    """split text to a set of words : bag-of-words"""
    mots = nltk.word_tokenize(phrases) 
    return mots 

def no_punctuation(mots):
    """return the set of words without puntctuation marks """
    regex =  re.compile('[%s]' % re.escape(string.punctuation))
    mots_no_punctuation = []
    for token in mots: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            mots_no_punctuation.append(new_token.lower()) #lower pour normaliser le texte
    return mots_no_punctuation

def delete_stop_words(mots):
    """return the set of words without insignifiant words like : le , de, ces, the... 
    """
    stops= set(stopwords.words('french')+stopwords.words('english'))
    mots_no_stopwords = []
    for mot in mots:
        if not mot in stops:
                mots_no_stopwords.append(mot)
    return mots_no_stopwords


    
def nettoyer_texte(text, lemma = False, Stemm = False ): 
    text_chunk = chunk_mot(text)
    text_punct = no_punctuation(text_chunk)
    text_f = delete_stop_words(text_punct)
    #if lemma == True :
        #text_f = word_lemmatizer(text_f)
    #if Stemm == True : 
        #text_f = word_stemmer(text_f)
    #reconstruct the text
    textn = ' '.join(text_f)
    return textn

In [6]:
all_news["title"]=all_news["title"].apply(lambda x : clean_text(x))
all_news["text"]=all_news["text"].apply(lambda x : clean_text(x))

all_news["title"]=all_news["title"].apply(lambda x : nettoyer_texte(x))
all_news["text"]=all_news["text"].apply(lambda x : nettoyer_texte(x))

LookupError: 
**********************************************************************
  Resource u'tokenizers/punkt/english.pickle' not found.  Please
  use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - '/home/paul/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - u''
**********************************************************************

In [ ]:
all_news.head()

- ### Création de la matrice

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from __future__ import print_function

import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt


from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics


In [ ]:
tokenize = lambda doc: doc.split(" ")
sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize) 
tfidf_title = sklearn_tfidf.fit_transform(all_news['title']) 
feature_names = sklearn_tfidf.get_feature_names()

In [ ]:
tfidf_title.shape

In [ ]:
ch2 = SelectKBest(chi2, k=100)
tfidf_title = ch2.fit_transform(tfidf_title,all_news['target'])
feature_names = [feature_names[i] for i
                         in ch2.get_support(indices=True)]


In [ ]:
feature_names


In [7]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True